In [12]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

from functools import partial
import matplotlib.pyplot as plt
import pandas as pd
import jax.random as random
import numpyro

import tarea

rng_key = random.PRNGKey(12345)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# ¿Es posible explicar la cantidad de billonarios en base al desarrollo país?  <a class="tocSkip"></a>







En 2006 *Daniel Treisman* publicó un artículo titulado [*Russia Billionaries*](https://pubs.aeaweb.org/doi/pdfplus/10.1257/aer.p20161068) en el cual conectó la cantidad de billonarios de un país con ciertos atributos económicos de los mismos. 

Su conclusión principal fue que Rusia tiene una cantidad de billonarios mayor que la que predicen los indicadores económicos

En esta tarea ustedes analizarán datos macroeconómicos para comprobar o refutar los hallazgos de *D. Treisman*

## Datos

Para esta tarea se les provee de un conjunto de datos `billonarios.csv` indexado por país con los siguientes atributos

- `nbillonarios`: La cantidad de billonarios del pais
- `logpibpc`: El logaritmo del Producto Interno Bruto (PIB) per capita del pais
- `logpob`: El logaritmo de la población del pais
- `gatt`: La cantidad de años que el pais está adherido al *General Agreement on Tariffs and Trade* (GATT)

In [3]:
df = pd.read_csv('data/billonarios.csv', index_col='pais')
display(df.head(5))
y = df["nbillonarios"].values
x  = df.drop("nbillonarios", axis=1).values
display(y.shape, x.shape)

,nbillonarios,logpibpc,logpob,gatt
pais,,,,
United States,469,10.786021,19.532846,60
Canada,25,10.743365,17.319439,0
"Bahamas, The",0,10.072139,12.760934,0
Aruba,0,10.223734,11.526276,0
Bermuda,0,11.446847,11.086334,0


(197,)

(197, 3)

## Modelo 

El objetivo principal de esta tarea es entrenar un modelo de regresión que prediga la cantidad de billonarios en función de los demás atributos

> El número de billonarios es una variable entera y no-negativa. 

Un modelo apropiado en este caso es la [regresión de Poisson](https://en.wikipedia.org/wiki/Poisson_distribution), donde definimos la probabilidad condicional para un pais $i$ como  

$$
p(y_i | x_i ) = \frac{\lambda_i^{y_i}}{y_i!} \exp \left ({-\lambda_i} \right)
$$

con intensidad

$$
\lambda_i = \exp \left (\theta_0 + \sum_{j=1}^M \theta_j x_{ij} \right)
$$

donde 

- $\theta$ es el vector de parámetros que deseamos ajustar 
- $y_i$ y $x_i$ son la cantidad de billonarios y el vector de atributos del país $i$, respectivamente


## Actividades

- Implemente el modelo usando numpyro (función `model` en `tarea.py`)
    - Considere un prior normal para los parámetros $\theta$
    - Considere una verosimilitud poisson
- Obtenga muestras del posterior utilizando MCMC
    - Muestre las trazas de los parámetros (utilize `matplotlib`)
    - Diagnóstique la convergencia en base a las trazas, número de muestras efectivo y el estadístico de Gelman-Rubin
- Análisis los resultados obtenidos
    - Muestre el posterior de los parámetros obtenidos (utilize `matplotlib`), ¿Cúales son significativamente distintos de cero?
    - Prediga la cantidad de billonarios y la incertidumbre asociada de cada pais usando su modelo (posterior predictivo. 
    - Responda y discuta ¿Cuáles son los 5 países con mayor error entre la predicción y el valor real? ¿Cuáles países tienen un exceso de billonarios? ¿Cúales paises tienen menos billonarios de lo esperado? ¿Qué puede decir sobre Rusia? ¿Cuáles son los 5 paises donde el modelo está más inseguro de sus resultados?

In [190]:
seeded_model = numpyro.handlers.seed(tarea.model, random.PRNGKey(12345))
exec_trace = numpyro.handlers.trace(seeded_model).get_trace(x, y)
print(numpyro.util.format_shapes(exec_trace))

[1. 1. 1.]


NameError: name 'pk' is not defined

In [191]:
rng_key, rng_key_ = random.split(rng_key)

posterior_samples = tarea.run_mcmc_nuts(seeded_model, x, y, rng_key_)

print(posterior_samples.keys())

[603280156 445306386]
[1. 1. 1.]


NameError: name 'pk' is not defined